# Instrument Classification - Part 3 : Track Prediction

In [352]:
import numpy as np
import librosa, os, keras, time
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pydub.utils import make_chunks

In [284]:
def crop_song(dir,songs):
    predict = []
    pred_all = []
    even = True
    it = 0
    for f in os.listdir(dir):
        if (it == songs): break
        if (even): 
            even = False 
            continue
        else:
            print(f)
            even = True
            # downsampled to 22050Hz and downmixed to mono
            wav,sr = librosa.load(dir+f,mono=True,sr=22050)
            # normalized by the root mean square energy
            wav = wav / np.sqrt(np.mean(wav**2))
            # chunked to 1sec long snippets
            chunks = make_chunks(wav,sr)
            for wav in chunks:
                # transformed into mel-spectrograms with given traits
                mel = librosa.feature.melspectrogram(wav,n_mels=96,fmax=11025,n_fft=1024,hop_length=256,power=1)
                # decibel scaling
                mel = librosa.core.amplitude_to_db(mel)
                # zero padding when mel shape is not (96,87)
                pad = np.zeros((96,87))
                if (mel.shape != pad.shape):
                    pad[:mel.shape[0],:mel.shape[1]] = mel
                    mel = pad
                predict.append(mel)
        it = it + 1
        pred_all.append(predict)
        predict = []
    return pred_all

def decide(pred):
    pred_m = []
    pred_avg = []
    for song in pred:
        maxp, avgp = [0,0,0,0], [0,0,0,0]
        for tupl in song:
            for i in range(len(tupl)):
                if tupl[i]>maxp[i]: maxp[i] = tupl[i]
                avgp[i] += tupl[i]/len(song)
        #for i in range(4): avgp[i] = avgp[i]/4        
        pred_m.append(maxp)
        pred_avg.append(avgp)
    return pred_m, pred_avg
        
#def extract_instruments(prediction):
    # connect to_categorical to instrument labels

In [326]:
model3 = keras.models.load_model('ICmodel_2_mix_big_more.h5')

In [344]:
dir = './IRMAS/TestingData/Part2/'
songs = 3
predict = crop_song(dir,songs)

000015_more than a feeling - boston-1.wav
000015_more than a feeling - boston-2.wav
000015_more than a feeling - boston-3.wav


In [345]:
pred_all = []
for song in predict:
    new = np.expand_dims(np.array(song),-1)
    prediction = model3.predict(new)
    pred_all.append(prediction)
n, m = decide(pred_all)

print("Evaluation for gac-voi improved mix model")
print("Song with gac")
print("max:",n[1])
print("avg:",m[1])
print("Song with gac-voi")
print("max:",n[2])
print("avg:",m[2])

Evaluation for gac-voi improved mix model
Song with gac
max: [0.9424996, 0.37426513, 0, 0]
avg: [0.7425469096217837, 0.13330195205552237, 0, 0]
Song with gac-voi
max: [0.88801444, 0.75555193, 0, 0]
avg: [0.48426078110933307, 0.4318032398819924, 0, 0]


In [346]:
dir = './IRMAS/TestingData/YT/'
predict = crop_song(dir,1)
pred_all = []
for song in predict:
    new = np.expand_dims(np.array(song),-1)
    prediction = model3.predict(new)
    pred_all.append(prediction)
n, m = decide2(pred_all)

gac-voi_TheUnforgiven.wav


In [378]:
pia_thr = 0.5
gac_thr = 0.5
dru_thr = 0.5
voi_thr = 0.5

new_prediction = [[0]*len(instr)]*(len(prediction))

instr = ['gac','voi']
thr = [gac_thr,voi_thr]
bool = [False,False]

for i in range(len(voi)):
    for j in range(len(instr)):
        if i==0:
            if (prediction[i][j] < thr[j] and prediction[i+1][j] > thr[j]): new_prediction[i][j] = prediction[i+1][j]
            elif (prediction[i][j] > thr[j] and prediction[i+1][j] < thr[j]): new_prediction[i][j] = prediction[i+1][j]
            else: new_prediction[i][j] = prediction[i][j]
        elif i==len(voi):
            if (prediction[i][j] < thr[j] and prediction[i-1][j] > thr[j]): new_prediction[i][j] = prediction[i-1][j]
            elif (prediction[i][j] > thr[j] and prediction[i-1][j] < thr[j]): new_prediction[i][j] = prediction[i-1][j]
            else: new_prediction[i][j] = prediction[i][j]
        else:
            if (prediction[i][j]<thr[j] and prediction[i-1][j]>thr[j] and prediction[i+1][j]>thr[j]):
                new_prediction[i][j] = (prediction[i-1][j]+prediction[i+1][j])/2
            elif (prediction[i][j]>thr[j] and prediction[i-1][j]<thr[j] and prediction[i+1][j]<thr[j]):
                new_prediction[i][j] = (prediction[i-1][j]+prediction[i+1][j])/2
            else: new_prediction[i][j] = prediction[i][j]
        if (new_prediction[i][j]>thr[j]): bool[j] = True

print("The track contains: ",end=" ")
for i in range(len(bool)):
    if (bool[i]): print(instr[i],end=" ")

print("\nBegins in 5 seconds")
time.sleep(1)
print("4")
time.sleep(1)
print("3")
time.sleep(1)
print("2")
time.sleep(1)
print("1")
time.sleep(1)
print("Go!")
for i in range(len(voi)):
    time.sleep(1)  
    print(str(i)+"-"+str(i+1)+" sec: ", end =" ")
    for j in range(len(instr)):
        if (prediction[i][j]>thr[j]): print(instr[j],round(prediction[i][j],2), end =" ")
    print("")

#t = np.arange(0,len(gac_new))
#plt.figure(figsize=(50,10))
#plt.plot(t[:150],gac_new[:150],voi_new[:150])

The track contains:  gac voi 
Begins in 5 seconds
4
3
2
1
Go!
0-1 sec:  gac 0.64 
1-2 sec:  gac 0.64 
2-3 sec:  gac 0.97 
3-4 sec:  gac 0.99 
4-5 sec:  gac 0.89 
5-6 sec:  gac 1.0 
6-7 sec:  gac 0.99 
7-8 sec:  gac 0.9 
8-9 sec:  gac 1.0 
9-10 sec:  gac 0.99 
10-11 sec:  gac 1.0 
11-12 sec:  gac 0.98 
12-13 sec:  gac 0.99 
13-14 sec:  gac 0.99 
14-15 sec:  gac 0.97 
15-16 sec:  gac 0.97 
16-17 sec:  gac 0.99 
17-18 sec:  gac 0.67 
18-19 sec:  gac 0.94 
19-20 sec:  gac 0.65 
20-21 sec:  gac 0.9 
21-22 sec:  gac 1.0 
22-23 sec:  gac 0.98 
23-24 sec:  gac 0.99 
24-25 sec:  gac 0.9 
25-26 sec:  gac 0.76 
26-27 sec:  gac 0.91 
27-28 sec:  gac 0.84 
28-29 sec:  gac 0.74 
29-30 sec:  gac 0.96 
30-31 sec:  gac 0.94 
31-32 sec:  gac 0.61 
32-33 sec:  gac 0.92 
33-34 sec:  gac 0.73 
34-35 sec:  gac 0.98 
35-36 sec:  gac 0.77 
36-37 sec:  gac 0.82 
37-38 sec:  gac 0.89 
38-39 sec:  gac 0.99 
39-40 sec:  gac 0.95 
40-41 sec:  gac 0.95 
41-42 sec:  gac 0.95 
42-43 sec:  gac 0.98 
43-44 sec:  gac 0.